In [65]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import pprint as pprint
import json


# Import API key
from api_keys import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)


In [66]:
wine_location = pd.read_csv("../Output/wine_cleaned.csv")
wine_location.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,Wine_Year
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012


In [67]:
#Add the latitude  and longitude  empty fields to prime for geocode info
wine_location['latitude'] = ""
wine_location['longitude'] = ""
wine_location.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,Wine_Year,latitude,longitude
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,,
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,,
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,,
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,,
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,,


In [68]:
wine_location.dtypes

country                   object
description               object
points                     int64
price                    float64
province                  object
region_1                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
Wine_Year                  int64
latitude                  object
longitude                 object
dtype: object

In [69]:
#Get the lat and long for the API

# Build the endpoint URL
geo_url = 'https://maps.googleapis.com/maps/api/geocode/json?address={0}&key={1}'

#Need the province , country from the table and combine together
#Create loop to gather the information

print("Beginning Data Retrieval")
print("-----------------------------")
set=1
# Loop through the list of province & country and perform a request for data on each
for index, row in wine_location.iterrows():
    
    #Get the fields from the table
    province_name = row['province']
    country_name = row['country']

    location = (province_name) + ", " + country_name
    #print(combo)
    geo_response = requests.get(geo_url.format(location, g_key)).json()
    #                            & province_name & ", " & country & "&key=" g_key).json()
    #print(geo_response)
    #print(json.dumps(geo_response, indent=4, sort_keys=True))
    try:
        # Extract latitude and longitude
        
        #print(geo_response["results"][0]["geometry"]["location"]["lat"])
        #print(geo_response["results"][0]["geometry"]["location"]["lng"])

        wine_location.loc[index, "latitude"] = geo_response["results"][0]["geometry"]["location"]["lat"]
        wine_location.loc[index, "longitude"] = geo_response["results"][0]["geometry"]["location"]["lng"]
        
        if index % 1000 == 0:
            print(f"Processing Record {index} of location {province_name}, {country_name}.")
        
    except (KeyError):
        print(f"{province_name}, {country_name} not found for Record {index} . Skipping...")
        set+= 1
    except (IndexError):
        print(f"Location not found has Index error {index} for {province_name}, {country_name}. Skipping...")
        set+= 1




Beginning Data Retrieval
-----------------------------
Processing Record 0 of location Sicily & Sardinia, Italy.
Location not found has Index error {index} for {province_name}, {country_name}. Skipping...
Location not found has Index error {index} for {province_name}, {country_name}. Skipping...
Location not found has Index error {index} for {province_name}, {country_name}. Skipping...
Location not found has Index error {index} for {province_name}, {country_name}. Skipping...
Location not found has Index error {index} for {province_name}, {country_name}. Skipping...
Location not found has Index error {index} for {province_name}, {country_name}. Skipping...
Location not found has Index error {index} for {province_name}, {country_name}. Skipping...
Location not found has Index error {index} for {province_name}, {country_name}. Skipping...
Location not found has Index error {index} for {province_name}, {country_name}. Skipping...
Location not found has Index error {index} for {province_na

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [70]:
wine_location.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,Wine_Year,latitude,longitude
0,Italy,"Aromas include tropical fruit, broom, brimston...",87,NaN,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013,37.6,14.0154
1,Portugal,"This is ripe and fruity, a wine that is smooth...",87,15.0,Douro,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011,41.5013,-5.51229
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,43.8041,-120.554
3,US,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Michigan,Lake Michigan Shore,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013,44.3148,-85.6024
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,43.8041,-120.554


In [71]:

wine_location = wine_location.replace('', np.nan)
wine_location = wine_location.dropna()
wine_location.head()

,country,description,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery,Wine_Year,latitude,longitude
2,US,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013,43.804133,-120.554201
4,US,"Much like the regular bottling from 2012, this...",87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012,43.804133,-120.554201
5,Spain,Blackberry and raspberry aromas show a typical...,87,15.0,Northern Spain,Navarra,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem,2011,43.336607,-3.202718
6,Italy,"Here's a bright, informal red that opens with ...",87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo,2013,37.599994,14.015356
7,France,This dry and restrained wine offers spice in p...,87,24.0,Alsace,Alsace,Roger Voss,@vossroger,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,Trimbach,2012,48.318179,7.441624


In [87]:
# Store latitude and longitude in locations
locations = wine_location[["latitude", "longitude"]]

points = wine_location["points"]

from ipywidgets.embed import embed_minimal_html

In [109]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")
#print(fig)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=points, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
#print(heat_layer)

# Customize the size of the figure


figure_layout = {
    'width': '1500px',
    'height': '700px',
    'border': '1px solid black',
    'padding': '12px',
    'margin': '0 auto 0 auto'
}

#1500 x 2100
fig = gmaps.figure(layout=figure_layout)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(border='1px solid black', height='700px', margin='0 auto 0 auto', padding='12px', w…

In [104]:
# Save the figure in html
embed_minimal_html('WinePoints1.html', views=[fig])